In [1]:
import base64
import requests
import datetime
from urllib.parse import urlencode

In [2]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    
    def get_client_credentials(self):
        """
        Return a base64 encoded string
        """        
        client_id = self.client_id
        client_secret = self.client_secret
        if (client_id == None or client_secret == None):
            raise Exception("Set client_id and client_server.")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
        
        
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {"Authorization":f"Basic {client_creds_b64}"}
    
    
    def get_token_data(self):
        return {"grant_type":"client_credentials"}
    
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if (r.status_code not in range(200,299)):
            raise Exception("Could not authenticate client.")
        token_response_data = r.json()
        now = datetime.datetime.now()
        access_token = token_response_data['access_token']
        expires_in = token_response_data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now   
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if (expires < now):
            self.perform_auth()
            return self.get_access_token()
        elif (token == None):
            self.perform_auth()
            return self.get_access_token()            
        return token
    
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    
    def get_resource(self, lookup_id, resource_type="albums", version="v1"):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"       
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        print(endpoint)
        print(headers)
        print(r)
        if (r.status_code not in range(200,299)):
            return {}
        return r.json()
    
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
        pass
        
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
        pass    
    
    def base_search(self, query_params):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"   
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(url=lookup_url, headers=headers)    
        if (r.status_code not in range(200,299)):
            return {}
        return r.json()
    
    def search(self, query=None, search_type='artist'):
        if query == None:
            raise Exception("A query is required.")
        if isinstance(query, dict):           
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        query_params = urlencode({"q": query,"type": search_type})
        return self.base_search(query_params)

In [3]:
CLIENT_ID = '2490920ce5574a1a9b97a3e366c39dd3'
CLIENT_SECRET = 'bfb4c438f0e742e3ab325f98a4b3b9dc'

spotify = SpotifyAPI(CLIENT_ID, CLIENT_SECRET)

In [8]:
query_params = {"track":"Time", "artist":"Hans"}
spotify.search(query_params, search_type="track")

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3ATime+artist%3AHans&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0YC192cP3KPCRWx8zr8MfZ'},
       'href': 'https://api.spotify.com/v1/artists/0YC192cP3KPCRWx8zr8MfZ',
       'id': '0YC192cP3KPCRWx8zr8MfZ',
       'name': 'Hans Zimmer',
       'type': 'artist',
       'uri': 'spotify:artist:0YC192cP3KPCRWx8zr8MfZ'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HR',
      'HU',
      'ID',
      'IE',
      'IL',
 

In [3]:
#spotify.get_artist('6fOMl44jA4Sp5b9PpYCkzz')